In [1]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to c:\users\mcagl\appdata\local\temp\pip-req-build-woqk5eqb
  Resolved https://github.com/openai/whisper.git to commit eff383b27b783e280c089475852ba83f20f64998
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\mcagl\AppData\Local\Temp\pip-req-build-woqk5eqb'


In [2]:
# import libraries
import os
import numpy as np
import pandas as pd
import whisper

from tqdm import tqdm

In [3]:
model = whisper.load_model("base")

In [11]:
# get audio clips and transcriptions
import os

recs = {}

for path, _, files in os.walk('./darpa-timit/data/TEST/'):
    for file in files:
        if str(file).endswith(".wav"):
            f = open(os.path.join(path, file).replace('\\', '/').replace('WAV.wav', 'TXT'))
            txt = f.readline()
            recs[os.path.join(path, file).replace('\\', '/')] = txt[8:-1]

recs

{'./darpa-timit/data/TEST/DR1/FAKS0/SA1.WAV.wav': 'She had your dark suit in greasy wash water all year.',
 './darpa-timit/data/TEST/DR1/FAKS0/SA2.WAV.wav': "Don't ask me to carry an oily rag like that.",
 './darpa-timit/data/TEST/DR1/FAKS0/SI1573.WAV.wav': 'His captain was thin and haggard and his beautiful boots were worn and shabby.',
 './darpa-timit/data/TEST/DR1/FAKS0/SI2203.WAV.wav': 'The reasons for this dive seemed foolish now.',
 './darpa-timit/data/TEST/DR1/FAKS0/SI943.WAV.wav': 'Production may fall far below expectations.',
 './darpa-timit/data/TEST/DR1/FAKS0/SX133.WAV.wav': 'Pizzerias are convenient for a quick lunch.',
 './darpa-timit/data/TEST/DR1/FAKS0/SX223.WAV.wav': 'Put the butcher block table in the garage.',
 './darpa-timit/data/TEST/DR1/FAKS0/SX313.WAV.wav': 'Drop five forms in the box before you go out.',
 './darpa-timit/data/TEST/DR1/FAKS0/SX403.WAV.wav': 'Her wardrobe consists of only skirts and blouses.',
 './darpa-timit/data/TEST/DR1/FAKS0/SX43.WAV.wav': 'Elde

In [16]:
# choose 100 random clips
import random

clips = []
for i in range(100):
    clips.append(random.choice(list(recs.items())))

clips

[('./darpa-timit/data/TEST/DR4/FRNG0/SI1985.WAV.wav',
  'Eyes like hot honey, eyes that sizzled.'),
 ('./darpa-timit/data/TEST/DR4/FJMG0/SI1181.WAV.wav',
  'Properly used, the present book is an excellent instrument of enlightenment.'),
 ('./darpa-timit/data/TEST/DR5/FHES0/SA2.WAV.wav',
  "Don't ask me to carry an oily rag like that."),
 ('./darpa-timit/data/TEST/DR2/MRCZ0/SX191.WAV.wav',
  'Withdraw all phony accusations at once.'),
 ('./darpa-timit/data/TEST/DR5/FASW0/SA2.WAV.wav',
  "Don't ask me to carry an oily rag like that."),
 ('./darpa-timit/data/TEST/DR5/FMAH0/SI1289.WAV.wav',
  'A flame would use up air.'),
 ('./darpa-timit/data/TEST/DR5/MRJM3/SI2078.WAV.wav',
  'Or the bay of female dogs in heat.'),
 ('./darpa-timit/data/TEST/DR3/MMWH0/SX369.WAV.wav',
  'Bob papered over the living room murals.'),
 ('./darpa-timit/data/TEST/DR1/MWBT0/SI923.WAV.wav',
  'To many experts, this trend was inevitable.'),
 ('./darpa-timit/data/TEST/DR5/FMAH0/SX389.WAV.wav',
  'They own a big house

In [18]:
# get results
results = []
for clip in tqdm(clips):
    result = model.transcribe(clip[0], **dict(without_timestamps=True))["text"]
    results.append(result)

results

100%|██████████| 100/100 [04:34<00:00,  2.75s/it]


[' Eyes like hot honey eyes that sizzled',
 ' Properly used, the present book is an excellent instrument of enlightenment.',
 " Don't ask me to carry an Oli rag like that.",
 ' with draw all 40 accusations at once.',
 " Don't ask me to carry an olive rag like that.",
 ' The flame would use up air.',
 ' are the Bay of Female Dogs in Heat.',
 ' Pop papered over the living room murals.',
 ' To many experts, this trend was inevitable.',
 ' They own a big house in the remote countryside.',
 ' Bricks are an alternative.',
 ' Where were you while we were away?',
 ' She had your dark suit and greasy wash water all year.',
 ' In short, scientific sampling was introduced in place of subjective sampling.',
 " Yes, because you didn't run off to use that new toothbrush.",
 " Before Thursday's exam, review every formula.",
 ' Progress is being made to an improving motorist access to many turnplikes.',
 ' Corian Trish played tag with beach balls for hours.',
 ' Should she wake him?',
 ' the causeway 

In [20]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

data = pd.DataFrame(dict(transcription=results, reference=[clips[i][1] for i in range(len(clips))]))
data["transcription"] = [normalizer(text) for text in data["transcription"]]
data["reference"] = [normalizer(text) for text in data["reference"]]
data

,transcription,reference
0,eyes like hot honey eyes that sizzled,eyes like hot honey eyes that sizzled
1,properly used the present book is an excellent...,properly used the present book is an excellent...
2,do not ask me to carry an oli rag like that,do not ask me to carry an oily rag like that
3,with draw all 40 accusations at once,withdraw all phony accusations at once
4,do not ask me to carry an olive rag like that,do not ask me to carry an oily rag like that
...,...,...
95,by that one feels that magnetic forces are as ...,by that one feels that magnetic forces are as ...
96,the cats me you all always hurts my ears,the cat is meow always hurts my ears
97,my father ran him off here 6 years ago,my father ran him off here 6 years ago
98,big that card for one gallon of gas,beg that guard for one gallon of gas


In [21]:
! pip install jiwer

In [22]:
import jiwer

wer = jiwer.wer(list(data["reference"]), list(data["transcription"]))

print(f"WER: {wer * 100:.2f} %")

WER: 8.30 %
